In [2]:
from platform import python_version

print(python_version())

3.7.4


In [4]:
jupyter kernelspec list

SyntaxError: invalid syntax (<ipython-input-4-68dcfb427789>, line 1)

In [3]:
import numpy as np
import pandas as pd
import os
import urllib
import matplotlib.pyplot as plt
import cv2
import matplotlib.patches as patches
import tensorflow as tf
import keras

from keras.utils import to_categorical
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential        # keras is only compatible with python 3.6 or lower
import sklearn
from sklearn.preprocessing import LabelEncoder
from mtcnn.mtcnn import MTCNN
import seaborn as sns

%matplotlib inline
sns.set_style("darkgrid")

Using TensorFlow backend.


In [4]:
NOTEBOOK_DIR = !pwd
NOTEBOOK_DIR = NOTEBOOK_DIR[0]
ROOT_DIR = os.path.abspath(os.path.join(NOTEBOOK_DIR, os.pardir))
DATA_DIR = os.path.join(ROOT_DIR, 'data')
IMAGES_DIR = os.path.join(DATA_DIR, 'images')
ANNOTATIONS_DIR = os.path.join(DATA_DIR, 'annotations')
SUB_IMAGES_DIR = os.path.join(IMAGES_DIR, 'sub')
MODEL_DIR = os.path.join(ROOT_DIR, 'models')
MNM_MODEL_DIR = os.path.join(MODEL_DIR, 'keras_mask_or_no_mask_model')


In [5]:
!pwd

/Users/kellymcgaw/Desktop/AI_class/face-mask-detection/notebooks


In [6]:
import cv2
import os
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np
 
# cascPath = os.path.dirname(
#     cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
# faceCascade = cv2.CascadeClassifier(cascPath)
model = load_model(MNM_MODEL_DIR)
detector = MTCNN()
video_capture = cv2.VideoCapture(0)
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     faces = faceCascade.detectMultiScale(gray,
#                                          scaleFactor=1.1,
#                                          minNeighbors=5,
#                                          minSize=(60, 60),
#                                          flags=cv2.CASCADE_SCALE_IMAGE)
    faces = detector.detect_faces(frame)
    faces_list=[]
    preds=[]
    faces = detector.detect_faces(frame)
    if faces != []:
        data = list()
        results = list()
        for face in faces:
            bounding_box = face['box']
            keypoints = face['keypoints']
            data.append([count, bounding_box])
                
            x, y, w, h = bounding_box
            img = frame[y:y+h,x:x+w]
            if img != []:
                new_img = cv2.resize(img,(50,50))
                new_img = new_img.reshape(-1,50,50,1)
                prediction = model.predict(new_img)
                print("PREDICTION: {}".format(prediction))
                results.append([count, bounding_box, prediction])
                cv2.rectangle(frame,
                              (x, y),
                              (x+w, y + h),
                              (0,155,255),
                              2)

                cv2.circle(frame,(keypoints['left_eye']), 2, (0,155,255), 2)
                cv2.circle(frame,(keypoints['right_eye']), 2, (0,155,255), 2)
                cv2.circle(frame,(keypoints['nose']), 2, (0,155,255), 2)
                cv2.circle(frame,(keypoints['mouth_left']), 2, (0,155,255), 2)
                cv2.circle(frame,(keypoints['mouth_right']), 2, (0,155,255), 2)
        # Display the resulting frame
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) &0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

NameError: name 'count' is not defined